# Investigation: Matrix Multiply

**From The Nature of Fast, Chapter 10**

Two matrix multiply implementations. Same algorithm. Same hardware. Same inputs.
One is 200× faster. This notebook shows you why.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ttsugriy/performance-book/blob/main/notebooks/tier2-experimental/10-matrix-multiply.ipynb)

---

## Setup

In [ ]:
import numpy as np
import time
import platform
import matplotlib.pyplot as plt

# Try to import numba for JIT compilation
try:
    from numba import jit, prange
    HAS_NUMBA = True
    print("✓ Numba available for JIT compilation")
except ImportError:
    HAS_NUMBA = False
    print("⚠️ Numba not available. Install with: pip install numba")
    print("   (Some benchmarks will be slow)")

print(f"\nPython: {platform.python_version()}")
print(f"NumPy: {np.__version__}")
print(f"Platform: {platform.platform()}")

## The Problem

Multiply two 1024×1024 matrices. Simple, right?

In [ ]:
# Setup: Create test matrices
N = 512  # Using 512 to keep demo fast; the book uses 2048
A = np.random.randn(N, N).astype(np.float32)
B = np.random.randn(N, N).astype(np.float32)
C = np.zeros((N, N), dtype=np.float32)

print(f"Matrix size: {N}×{N}")
print(f"Memory per matrix: {N * N * 4 / 1e6:.1f} MB")
print(f"Total FLOPs: {2 * N**3 / 1e9:.2f} billion")

## Implementation 1: Triple Nested Loop (Naive)

The textbook algorithm: C[i,j] = Σ_k A[i,k] × B[k,j]

In [ ]:
def matmul_naive_python(A, B, C):
    """Pure Python triple loop - will be very slow!"""
    M, K = A.shape
    K, N = B.shape
    for i in range(M):
        for j in range(N):
            total = 0.0
            for k in range(K):
                total += A[i, k] * B[k, j]
            C[i, j] = total
    return C

if HAS_NUMBA:
    @jit(nopython=True)
    def matmul_naive(A, B, C):
        """Naive i-j-k loop order."""
        M, K = A.shape
        K, N = B.shape
        for i in range(M):
            for j in range(N):
                total = 0.0
                for k in range(K):
                    total += A[i, k] * B[k, j]
                C[i, j] = total
        return C
else:
    matmul_naive = matmul_naive_python

# Warmup and test
C_naive = np.zeros((N, N), dtype=np.float32)
if N <= 256 or HAS_NUMBA:  # Only run if fast enough
    _ = matmul_naive(A[:64, :64], B[:64, :64], np.zeros((64, 64), dtype=np.float32))
    print("Naive implementation ready")
else:
    print("⚠️ Skipping naive warmup (too slow without Numba)")

## Implementation 2: Reordered Loops (i-k-j)

**Key insight**: The order of loops matters for memory access!

- Naive (i-j-k): Accesses B column-wise (strided access)
- Reordered (i-k-j): Accesses B row-wise (sequential access)

In [ ]:
if HAS_NUMBA:
    @jit(nopython=True)
    def matmul_reordered(A, B, C):
        """Reordered i-k-j loop - better memory access!"""
        M, K = A.shape
        K, N = B.shape
        C[:] = 0  # Reset
        for i in range(M):
            for k in range(K):
                a_val = A[i, k]  # Load once
                for j in range(N):
                    C[i, j] += a_val * B[k, j]  # Sequential access!
        return C
    
    # Warmup
    _ = matmul_reordered(A[:64, :64], B[:64, :64], np.zeros((64, 64), dtype=np.float32))
    print("Reordered implementation ready")
else:
    print("⚠️ Reordered version requires Numba")
    matmul_reordered = None

## Implementation 3: Tiled (Cache-Friendly)

Process in blocks that fit in cache!

In [ ]:
if HAS_NUMBA:
    @jit(nopython=True)
    def matmul_tiled(A, B, C, tile_size=32):
        """Tiled matrix multiply for cache efficiency."""
        M, K = A.shape
        K, N = B.shape
        C[:] = 0
        
        for i0 in range(0, M, tile_size):
            for j0 in range(0, N, tile_size):
                for k0 in range(0, K, tile_size):
                    # Process one tile
                    i_end = min(i0 + tile_size, M)
                    j_end = min(j0 + tile_size, N)
                    k_end = min(k0 + tile_size, K)
                    
                    for i in range(i0, i_end):
                        for k in range(k0, k_end):
                            a_val = A[i, k]
                            for j in range(j0, j_end):
                                C[i, j] += a_val * B[k, j]
        return C
    
    # Warmup
    _ = matmul_tiled(A[:64, :64], B[:64, :64], np.zeros((64, 64), dtype=np.float32))
    print("Tiled implementation ready")
else:
    print("⚠️ Tiled version requires Numba")
    matmul_tiled = None

## Implementation 4: NumPy (BLAS)

NumPy uses highly optimized BLAS libraries (OpenBLAS, MKL).

In [ ]:
def matmul_numpy(A, B, C):
    """Use NumPy's optimized matrix multiply."""
    np.matmul(A, B, out=C)
    return C

# Warmup
_ = matmul_numpy(A, B, C.copy())
print("NumPy/BLAS implementation ready")

## Benchmark: The Moment of Truth

In [ ]:
def benchmark(fn, A, B, C, num_runs=3, name=""):
    """Benchmark a matrix multiply function."""
    times = []
    for _ in range(num_runs):
        C_test = C.copy()
        start = time.perf_counter()
        fn(A, B, C_test)
        elapsed = time.perf_counter() - start
        times.append(elapsed)
    
    mean_time = np.mean(times)
    flops = 2 * A.shape[0] * A.shape[1] * B.shape[1]
    gflops = flops / mean_time / 1e9
    
    print(f"{name:20s}: {mean_time:8.4f}s ({gflops:6.2f} GFLOPS)")
    return mean_time, gflops

print(f"Benchmarking {N}×{N} matrix multiply...\n")
print("=" * 50)

results = {}

# NumPy first (always available)
t, gf = benchmark(matmul_numpy, A, B, C.copy(), name="NumPy (BLAS)")
results['NumPy'] = (t, gf)

if HAS_NUMBA:
    # Run our implementations
    t, gf = benchmark(matmul_tiled, A, B, C.copy(), name="Tiled (32×32)")
    results['Tiled'] = (t, gf)
    
    t, gf = benchmark(matmul_reordered, A, B, C.copy(), name="Reordered (i-k-j)")
    results['Reordered'] = (t, gf)
    
    if N <= 256:  # Only run naive for small matrices
        t, gf = benchmark(matmul_naive, A, B, C.copy(), name="Naive (i-j-k)")
        results['Naive'] = (t, gf)
    else:
        print(f"{'Naive (i-j-k)':20s}: (skipped - too slow for N={N})")

print("=" * 50)

## Analysis: Why Is There Such a Difference?

In [ ]:
# Visualize the results
if len(results) > 1:
    names = list(results.keys())
    gflops = [results[n][1] for n in names]
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(names, gflops, color=['green', 'blue', 'orange', 'red'][:len(names)])
    plt.ylabel('GFLOPS', fontsize=12)
    plt.title(f'Matrix Multiply Performance ({N}×{N})', fontsize=14)
    
    # Add value labels
    for bar, gf in zip(bars, gflops):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                 f'{gf:.1f}', ha='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate speedups
    baseline = results.get('Naive', results.get('Reordered', (1, 1)))[1]
    print("\nSpeedups relative to slowest:")
    for name, (t, gf) in sorted(results.items(), key=lambda x: x[1][1]):
        print(f"  {name}: {gf/baseline:.1f}×")

## Investigation: Why Does Loop Order Matter?

Let's visualize the memory access patterns.

In [ ]:
def visualize_access_pattern(loop_order, n=8):
    """
    Visualize which elements of B are accessed in sequence.
    """
    accesses = []
    
    if loop_order == 'ijk':  # Naive
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    accesses.append((k, j))  # B[k, j]
    elif loop_order == 'ikj':  # Reordered
        for i in range(n):
            for k in range(n):
                for j in range(n):
                    accesses.append((k, j))  # B[k, j]
    
    return accesses[:64]  # First 64 accesses

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, (order, title) in zip(axes, [('ijk', 'Naive (i-j-k): Column-major access'),
                                       ('ikj', 'Reordered (i-k-j): Row-major access')]):
    accesses = visualize_access_pattern(order)
    
    # Create grid
    grid = np.zeros((8, 8))
    for idx, (k, j) in enumerate(accesses):
        grid[k, j] = idx + 1
    
    im = ax.imshow(grid, cmap='YlOrRd')
    ax.set_title(title, fontsize=12)
    ax.set_xlabel('j (column)')
    ax.set_ylabel('k (row)')
    
    # Show access order for first few
    for idx, (k, j) in enumerate(accesses[:20]):
        ax.annotate(str(idx), (j, k), ha='center', va='center', fontsize=7)

plt.tight_layout()
plt.show()

print("\nNaive (left): Jumps between columns (stride = N)")
print("Reordered (right): Sequential within rows (stride = 1)")
print("\nSequential access is 5-10× faster due to caching!")

## Investigation: Optimal Tile Size

In [ ]:
if HAS_NUMBA:
    def test_tile_sizes():
        tile_sizes = [8, 16, 32, 64, 128]
        results = []
        
        for ts in tile_sizes:
            # Create tiled version with this tile size
            @jit(nopython=True)
            def matmul_tiled_ts(A, B, C, tile_size=ts):
                M, K = A.shape
                K, N = B.shape
                C[:] = 0
                for i0 in range(0, M, tile_size):
                    for j0 in range(0, N, tile_size):
                        for k0 in range(0, K, tile_size):
                            i_end = min(i0 + tile_size, M)
                            j_end = min(j0 + tile_size, N)
                            k_end = min(k0 + tile_size, K)
                            for i in range(i0, i_end):
                                for k in range(k0, k_end):
                                    a_val = A[i, k]
                                    for j in range(j0, j_end):
                                        C[i, j] += a_val * B[k, j]
                return C
            
            # Warmup
            _ = matmul_tiled_ts(A[:64, :64], B[:64, :64], np.zeros((64, 64), dtype=np.float32))
            
            # Benchmark
            times = []
            for _ in range(3):
                C_test = C.copy()
                start = time.perf_counter()
                matmul_tiled_ts(A, B, C_test)
                times.append(time.perf_counter() - start)
            
            mean_time = np.mean(times)
            flops = 2 * N**3
            gflops = flops / mean_time / 1e9
            
            working_set_kb = 3 * ts * ts * 4 / 1024
            results.append((ts, gflops, working_set_kb))
            print(f"Tile {ts:3d}×{ts:3d} (working set {working_set_kb:5.1f} KB): {gflops:.2f} GFLOPS")
        
        return results
    
    print(f"Testing tile sizes for {N}×{N} matrix...\n")
    tile_results = test_tile_sizes()
    
    # Visualize
    ts = [r[0] for r in tile_results]
    gf = [r[1] for r in tile_results]
    
    plt.figure(figsize=(10, 5))
    plt.bar([str(t) for t in ts], gf)
    plt.xlabel('Tile Size')
    plt.ylabel('GFLOPS')
    plt.title('Performance vs Tile Size')
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Optimal tile size depends on your cache hierarchy!")
    print("   L1: ~32KB, L2: ~256KB, L3: ~8-32MB (typical)")
else:
    print("⚠️ This investigation requires Numba")

## Correctness Check

Make sure all implementations compute the same result!

In [ ]:
# Ground truth
C_numpy = A @ B

# Test each implementation
if HAS_NUMBA:
    C_tiled = np.zeros_like(C)
    matmul_tiled(A, B, C_tiled)
    print(f"Tiled matches NumPy: {np.allclose(C_numpy, C_tiled)}")
    print(f"  Max difference: {np.max(np.abs(C_numpy - C_tiled)):.2e}")
    
    C_reordered = np.zeros_like(C)
    matmul_reordered(A, B, C_reordered)
    print(f"Reordered matches NumPy: {np.allclose(C_numpy, C_reordered)}")
    print(f"  Max difference: {np.max(np.abs(C_numpy - C_reordered)):.2e}")

## Your Turn: Experiments

1. **Matrix size**: How does performance scale with N?
2. **Tile size**: Find the optimal tile size for YOUR hardware.
3. **Data types**: Compare float32 vs float64 performance.

In [ ]:
# Your experiments here!

# Example: How does NumPy scale with matrix size?
def scaling_experiment():
    sizes = [64, 128, 256, 512, 1024]
    results = []
    
    for n in sizes:
        A = np.random.randn(n, n).astype(np.float32)
        B = np.random.randn(n, n).astype(np.float32)
        
        # Warmup
        _ = A @ B
        
        # Benchmark
        times = []
        for _ in range(3):
            start = time.perf_counter()
            _ = A @ B
            times.append(time.perf_counter() - start)
        
        mean_time = np.mean(times)
        flops = 2 * n**3
        gflops = flops / mean_time / 1e9
        
        results.append((n, gflops))
        print(f"N={n:5d}: {gflops:6.2f} GFLOPS")
    
    return results

print("NumPy scaling with matrix size:\n")
scaling = scaling_experiment()

## Key Takeaways

1. **Loop order matters**: i-k-j is ~4× faster than i-j-k due to memory access patterns.

2. **Tiling transforms memory-bound to compute-bound**: By keeping working set in cache.

3. **BLAS is highly optimized**: Uses multi-level tiling, SIMD, and parallelism.

4. **The same algorithm can be 200× slower**: Without attention to memory hierarchy.

5. **Hardware determines optimal tile size**: Match to your cache hierarchy.

---

*Continue to [Chapter 11: FlashAttention](https://ttsugriy.github.io/performance-book/chapters/11-flash-attention.html)*